In [3]:
from edgar import set_identity
import pandas as pd
import time
from edgar import *

In [4]:
# must be set to comply with SEC regulations

set_identity("Neal Lockhart nhl3388@uncw.edu")

[17:08:08] INFO     Identity of the Edgar REST client set to [Neal Lockhart nhl3388@uncw.edu]           ]8;id=62101;file:///home/neal/nealsfiles/stock_rag/stock_advisor_model/venv/lib/python3.10/site-packages/edgar/core.py\core.py]8;;\:]8;id=549466;file:///home/neal/nealsfiles/stock_rag/stock_advisor_model/venv/lib/python3.10/site-packages/edgar/core.py#161\161]8;;\

In [5]:
companies = ['HUBS', 'SNOW', 'CRWD', 'ZS', 'NET', 'AMD', 'NVDA', 'BASE', 'NOW', 'S']

In [6]:
tenks = Company('S').get_filings(form = '10-K').latest(2)

In [8]:
test = tenks[0].sections()

### Risks

In [16]:
def get_risk_section(companies: list):

    data = {
        "ticker": [],
        "chunked_risk": []
    }
    
    for company in companies:
        time.sleep(.1)
        tenk = Company(company).get_filings(form="10-K").latest(1)
        raw_chunked_tenk = tenk.sections()
        index_start = None
        index_end = None
        
        for i in range(len(raw_chunked_tenk)):
            if re.search(r"item 1a.\s*risk factors", raw_chunked_tenk[i].lower()):
                index_start=i
            elif re.search(r"item 1b.\s*unresolved", raw_chunked_tenk[i].lower()):
                index_end=i
                break

        if index_start is None or index_end is None:
            print("Warning: Could not find risk section for", company)
            continue
        
        risks = raw_chunked_tenk[index_start:index_end]

        for i in range(len(risks)):
            risks[i] = risks[i].lower()
            risks[i] = risks[i].replace("\n"," ")
            risks[i] = risks[i].replace("•", " ")
            risks[i] = risks[i].replace("table of contents","")
            
            data['ticker'].append(company)
            data['chunked_risk'].append(risks[i])

    return pd.DataFrame(data)

### Management's Discussion

In [40]:
def get_mgmt_discussion_section(companies: list):

    data = {
        "ticker": [],
        "chunked_mgmt_discussion": []
    }
    
    for company in companies:
        time.sleep(.1)
        tenk = Company(company).get_filings(form="10-K").latest(1)
        raw_chunked_tenk = tenk.sections()
        index_start = None
        index_end = None
        
        for i in range(len(raw_chunked_tenk)):
            if re.search(r"item 7.\s*management", raw_chunked_tenk[i].lower()):
                index_start=i
            elif re.search(r"item 7a.\s*(qualitative|quantitative)", raw_chunked_tenk[i].lower()):
                index_end=i
                break

        if index_start is None or index_end is None:
            print("Warning: Could not find management's discussion section for", company)
            continue
        
        mgmt_discussion = raw_chunked_tenk[index_start:index_end]

        for i in range(len(mgmt_discussion)):
            mgmt_discussion[i] = mgmt_discussion[i].lower()
            mgmt_discussion[i] = mgmt_discussion[i].replace("\n"," ")
            mgmt_discussion[i] = mgmt_discussion[i].replace("•", " ")
            mgmt_discussion[i] = mgmt_discussion[i].replace("table of contents","")
            mgmt_discussion[i] = " ".join(mgmt_discussion[i].split())
            
            data['ticker'].append(company)
            data['chunked_mgmt_discussion'].append(mgmt_discussion[i])

    return pd.DataFrame(data)